
# COVID-Team-1
### Dr. Mohanty
### Brantley, Kawabata, Urgitus
### CSC 405 01
### Last Updated: 10/13/2020
### We, the members of COVID-Team-1, have abided by the UNCG academic policy.

### Module Imports

In [1]:
import ipynb
from ipynb.fs.full import covid_data as cd
import pandas as pd

### Pandas Display Options

In [2]:
# Telling pandas not convert to html tags.
pd.set_option('display.notebook_repr_html', False)
# Max columns and rows to display.
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 10)

## Calculating Weekly Statistics

In [12]:
# Get covid daily cases.
df_daily_cases = cd.get_covid_daily_cases()
# Get primary data and group by state.
df_primary = cd.get_covid_primary_data()
df_primary_states = df_primary.groupby("State")
# Calculate the number of columns, i.e. days.
number_days = len(df_primary_states.first().filter(regex = "case").columns)
# Declaring/Initializing our state/week lists which will form our indexes.
state_index = []
week_index = []
# Keeps track of which week we are on.
week_count = 1
# Appends a state to the state index.
# Appends ((number_days) / 7) weeks into week index.
# Populating our lists, which will be used for multi indexing.
for state in df_daily_cases.index:
    for week in range (0, number_days, 7):
        state_index.append(state)
        week_index.append("week " + str(week_count))
        week_count += 1
    week_count = 1
# Creating our daily statistics dataframe for cases.
df_weekly_statistics_cases = pd.DataFrame(index = [state_index, week_index],\
                                   columns = ["mean", "median", "start_day"])
# Creating our daily statistics dataframe for deaths.
df_daily_deaths = cd.get_covid_daily_deaths()
df_weekly_statistics_deaths = pd.DataFrame(index = [state_index, week_index],\
                                   columns = ["mean", "median", "start_day"])
# Calculating weekly mean/median for cases and deaths.
# Week count starts at 0, it is now used for index positioning.
week_count = 0
for lower_range in range(0, number_days, 7):
    upper_range = lower_range + 7
    for state in df_daily_cases.index:
        # Get weekly statistics for cases.
        df_weekly_statistics_cases.loc[state].iloc[week_count]["mean"]\
            = df_daily_cases.loc[state].iloc[lower_range:upper_range].mean()
        df_weekly_statistics_cases.loc[state].iloc[week_count]["median"]\
            = df_daily_cases.loc[state].iloc[lower_range:upper_range].median()
        df_weekly_statistics_cases.loc[state].iloc[week_count]["start_day"]\
            = df_daily_cases.loc[state].index[lower_range]
        # Get weekly statistics for deaths.
        df_weekly_statistics_deaths.loc[state].iloc[week_count]["mean"]\
            = df_daily_deaths.loc[state].iloc[lower_range:upper_range].mean()
        df_weekly_statistics_deaths.loc[state].iloc[week_count]["median"]\
            = df_daily_deaths.loc[state].iloc[lower_range:upper_range].median()
        df_weekly_statistics_deaths.loc[state].iloc[week_count]["start_day"]\
            = df_daily_deaths.loc[state].index[lower_range]
    week_count += 1

### Weekly Statistics for Cases by State

In [6]:
display(df_weekly_statistics_cases)

               mean median        start_day
AK week 1         0      0  cases - 1/22/20
   week 2         0      0  cases - 1/29/20
   week 3         0      0   cases - 2/5/20
   week 4         0      0  cases - 2/12/20
   week 5         0      0  cases - 2/19/20
...             ...    ...              ...
WY week 31  38.8571     35  cases - 8/19/20
   week 32       33     25  cases - 8/26/20
   week 33  33.8571     31   cases - 9/2/20
   week 34       48     49   cases - 9/9/20
   week 35      128    128  cases - 9/16/20

[1785 rows x 3 columns]

### Weekly Statistics for Deaths by State

In [17]:
display(df_weekly_statistics_deaths.loc['AK'])

week 1            0
week 2            0
week 3            0
week 4            0
week 5            0
             ...   
week 31           1
week 32    0.428571
week 33    0.428571
week 34    0.285714
week 35           0
Name: mean, Length: 35, dtype: object

## Comparing Weekly Statistics

In [15]:
#us_cases = pd.read_csv('../data/Daily/daily_new_cases_normalized.csv')
world_cases = pd.read_csv('../data/Covid_International/new_cases_normalized.csv')

world_cases

           Date  Afghanistan  Albania  Algeria  Andorra  Angola  ...  \
0    2020-01-01            0        0        0        0       0  ...   
1    2020-01-02            0        0        0        0       0  ...   
2    2020-01-03            0        0        0        0       0  ...   
3    2020-01-04            0        0        0        0       0  ...   
4    2020-01-05            0        0        0        0       0  ...   
..          ...          ...      ...      ...      ...     ...  ...   
283  2020-10-10            3      580       33    16566      29  ...   
284  2020-10-11            0      573       31        0      22  ...   
285  2020-10-12           25      584       30        0      65  ...   
286  2020-10-13           33      594       35    38698      37  ...   
287  2020-10-14           17      632       40        0      37  ...   

     Venezuela  Vietnam  Western_Sahara  Yemen  Zambia  Zimbabwe  
0            0        0               0      0       0         0  
1